In [10]:
import pandas as pd

In [19]:
import opensmile
import os
import requests
import time
# https://github.com/audeering/opensmile-python
smile = opensmile.Smile(
    # feature_set=opensmile.FeatureSet.ComParE_2016,
    # feature_set=opensmile.FeatureSet.eGeMAPSv02,
    feature_set=opensmile.FeatureSet.emobase,
    feature_level=opensmile.FeatureLevel.Functionals,
)


def error(id, e):
    content = '[{}]\t{}'.format(str(id), str(e))
    with open(r'./audio_process_error.log', 'a') as f:
        f.write('{}\n'.format(content))


def sleep():
    # print('sleeping...')
    # print()
    # time.sleep(15*60)
    time.sleep(1)


def process(url, id):
    filename = './tmp/{}.mp3'.format(id)
    state = 1
    trytime = 3
    while trytime > 0 and state != 0:
        try:
            # audio = requests.get(url)
            with requests.Session() as s:
                audio = s.get(url)
            with open(filename, 'wb') as f:
                f.write(audio.content)
            state = 0
        except Exception as e:
            if 'Temporary failure in name resolution' in str(e):
                # print(id, e)
                error(id, e)
                sleep()
                break
            if 'timeout' not in str(e):
                # print(id, e)
                error(id, e)
                sleep()
                break
            # print(id, e)
            print(e)
            error(id, e)
            sleep()
            state = 1
            trytime -= 1
    try:
        if os.path.exists(filename):
            processed = smile.process_file(filename)
            os.remove(filename)
            processed['track_id'] = id
            # result = {'col':processed.columns.tolist(), 'data': processed.iloc[0].tolist()}
            processed.index = [0]
            # print(processed)
            return processed
        else:
            return None
    except Exception as e:
        # print(id, e)
        error(id, e)
        # raise e
    return None

def download_mp3(url, id_):
    filename = './tmp/{}.mp3'.format(id_)
    with requests.Session() as s:
        audio = s.get(url)
        with open(filename, 'wb') as f:
            f.write(audio.content)
    return {id_: filename}

def process(filename, id_):
    if os.path.exists(filename):
        processed = smile.process_file(filename)
        # os.remove(filename)
        processed['track_id'] = id
        processed.index = [0]
        return processed


In [12]:
songs = pd.read_csv(r'./HKU956/2. original_song_audio.csv')

In [13]:
from tqdm import tqdm

In [18]:
mp3s = {}
for row in tqdm(songs.iterrows()):
    id_ = row[1]['song_id']
    link = row[1]['link']
    mp3s.update(download_mp3(link, id_))

592it [09:36,  1.03it/s]


In [20]:
df = []
for id_, filename in mp3s.items():
    df.append(process(filename, id_))

FileNotFoundError: ffmpeg cannot be found.
Please make sure it is installed.
For further instructions visit: https://audeering.github.io/audiofile/installation.html